In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text

In [7]:
instruments = ['hc','rb','i','j','jm','au','ag','v','ru','l','pp','bu','TA','FG','MA',
               'y','p','m','a','c','cs','jd','RM','CF','SR','OI']
start_date = 20180101
table = "AdjustedFuturesDaily"
engine = create_engine("sqlite:///../data/FuturesMarketData.db")
in_binds = ", ".join([f":sym{i}" for i in range(len(instruments))])
sql = text(f"""
    SELECT *, (ClosePrice * factor_multiply) as adjclose
    FROM {table}
    WHERE TradingDay >= :start
    AND Instrument IN ({in_binds})
""")

params = {"start": start_date} | {f"sym{i}": s for i, s in enumerate(instruments)}

with engine.begin() as conn:
    df = pd.read_sql(sql, conn, params=params)

df = df.pivot(index="TradingDay", columns="Instrument")
df["adjclose"]


Instrument,CF,FG,MA,OI,RM,SR,TA,a,ag,au,...,jd,jm,l,m,p,pp,rb,ru,v,y
TradingDay,,,,,,,,,,,,,,,,,,,,,
20180102,14723.312083,2044.243353,3187.277950,5870.018972,3622.435790,3951.124672,6409.453051,4143.605484,3174.812591,248.445148,...,3682.631235,1337.485979,12780.084953,6227.348248,3739.096470,15452.700709,2716.422467,7786.682521,4861.770518,4510.188152
20180103,14644.996593,2033.057343,3172.012164,5889.771254,3631.935185,3956.484855,6381.865735,4166.378803,3170.744405,249.289897,...,3640.522146,1351.480600,12805.760663,6245.365892,3734.884195,15447.817311,2670.844909,7762.247324,4854.599765,4508.627534
20180104,14693.943774,2056.827614,3198.182082,5927.480155,3614.519628,3924.993781,6476.122398,4177.765463,3160.980758,248.089464,...,3633.822973,1366.474838,12825.017446,6238.609276,3760.157847,15503.162494,2677.856841,7664.506540,4894.038906,4544.521765
20180105,14850.574754,2041.446850,3188.368363,5950.823760,3630.351953,3931.023987,6471.524512,4181.181461,3165.048944,248.845292,...,3625.209750,1359.977335,12805.760663,6249.870303,3764.370123,15477.117702,2663.131784,7650.931431,4897.624283,4544.521765
20180108,14855.469472,2114.155916,3262.516465,5907.727873,3609.769931,3935.714147,6538.193859,4199.400116,3165.862581,249.156515,...,3557.260993,1389.965810,12786.503880,6231.852659,3726.459644,15511.301491,2677.155648,7642.786365,4897.624283,4491.460728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20191225,10235.219679,2450.390868,2075.510711,6421.548746,3854.585317,3716.075319,6669.186159,3314.283424,3308.878432,289.498504,...,3727.844501,1566.202538,10552.153301,6957.642422,3423.864766,15011.191123,3655.007318,4954.526716,5184.511302,4430.568014
20191226,10312.089523,2460.412917,2072.633386,6492.842705,3827.488057,3710.612511,6679.942911,3315.150583,3350.513828,289.784819,...,3717.689654,1553.281197,10595.223314,6942.417821,3448.757372,15150.960128,3701.153752,4950.662031,5196.484769,4456.201629
20191227,10488.890163,2477.116331,2106.202182,6576.018991,3852.891739,3754.314977,6776.753678,3322.087859,3327.046605,290.778502,...,3640.512813,1560.081903,10588.044979,6939.880388,3565.300026,15204.871030,3713.739143,4966.120772,5192.493614,4565.481778


In [3]:
px = df["adjclose"].sort_index()
px = px.where(px > -1)

L = 15
s = 1
logp = np.log(px)
signal = logp.shift(s) - logp.shift(L - s)
signal

Instrument,CF,FG,MA,OI,RM,SR,TA,a,ag,au,...,jd,jm,l,m,p,pp,rb,ru,v,y
TradingDay,,,,,,,,,,,,,,,,,,,,,
20180102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20180103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20180104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20180105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20180108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20191225,0.026507,0.045632,0.009938,0.022692,-0.004392,-0.000368,0.014203,-0.018225,0.023767,0.000498,...,-0.110926,-0.007433,0.012405,-0.001090,0.046016,-0.019551,0.030665,-0.029991,0.009183,0.043830
20191226,0.003385,0.036090,0.029178,0.026654,-0.002633,0.007377,0.017988,-0.015061,0.038747,0.013707,...,-0.117813,-0.007374,0.010944,-0.002550,0.036686,-0.021840,0.013869,-0.035253,0.003084,0.037857
20191227,0.014643,0.015048,0.024794,0.031336,-0.021450,-0.006786,0.014190,-0.008854,0.067781,0.022541,...,-0.072937,-0.021708,0.014330,-0.014514,0.029296,-0.006175,0.005114,-0.029991,-0.009935,0.035124


In [4]:
window = 15
trade_percent = 0.2
gross_target = 1.0
hold_period = 2
data = df["adjclose"]

low = data.quantile(trade_percent, axis=1)
high = data.quantile(1 - trade_percent, axis=1)

longs = (data.ge(high, axis=0)).astype(float)
shorts = (data.le(low, axis=0)).astype(float) * -1

weights = longs + shorts
weights = weights.sub(weights.mean(axis=1), axis=0)

gross = weights.abs().sum(axis=1)
scale = gross_target / gross.replace(0, np.nan)
position = weights.mul(scale, axis=0).fillna(0)
position

Instrument,CF,FG,MA,OI,RM,SR,TA,a,ag,au,...,jd,jm,l,m,p,pp,rb,ru,v,y
TradingDay,,,,,,,,,,,,,,,,,,,,,
20180102,0.083333,0.000000,0.000000,0.000000,0.0,0.0,0.083333,0.0,0.0,-0.083333,...,0.0,-0.083333,0.083333,0.083333,0.0,0.083333,0.0,0.083333,0.0,0.0
20180103,0.083333,0.000000,0.000000,0.000000,0.0,0.0,0.083333,0.0,0.0,-0.083333,...,0.0,-0.083333,0.083333,0.083333,0.0,0.083333,0.0,0.083333,0.0,0.0
20180104,0.083333,-0.083333,0.000000,0.000000,0.0,0.0,0.083333,0.0,0.0,-0.083333,...,0.0,-0.083333,0.083333,0.083333,0.0,0.083333,0.0,0.083333,0.0,0.0
20180105,0.083333,0.000000,0.000000,0.000000,0.0,0.0,0.083333,0.0,0.0,-0.083333,...,0.0,-0.083333,0.083333,0.083333,0.0,0.083333,0.0,0.083333,0.0,0.0
20180108,0.083333,0.000000,0.000000,0.000000,0.0,0.0,0.083333,0.0,0.0,-0.083333,...,0.0,-0.083333,0.083333,0.083333,0.0,0.083333,0.0,0.083333,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20191225,0.083333,0.000000,-0.083333,0.000000,0.0,0.0,0.083333,0.0,0.0,-0.083333,...,0.0,-0.083333,0.083333,0.083333,0.0,0.083333,0.0,0.000000,0.0,0.0
20191226,0.083333,0.000000,-0.083333,0.000000,0.0,0.0,0.083333,0.0,0.0,-0.083333,...,0.0,-0.083333,0.083333,0.083333,0.0,0.083333,0.0,0.000000,0.0,0.0
20191227,0.083333,0.000000,-0.083333,0.083333,0.0,0.0,0.083333,0.0,0.0,-0.083333,...,0.0,-0.083333,0.083333,0.083333,0.0,0.083333,0.0,0.000000,0.0,0.0


In [5]:
price_diff = data.diff(periods=1)
pos_shift = position.shift(1).fillna(0)
pnl = pos_shift * price_diff

# Portfolio PnL
pnl_ptf = pnl.sum(axis=1)

# Turnover (sum of absolute position changes)
turnover = (position - pos_shift).abs().sum(axis=1)

In [6]:
open_interest = df["OpenInterest"]
adjusted_close = df["adjclose"]

open_interest

Instrument,CF,FG,MA,OI,RM,SR,TA,a,ag,au,...,jd,jm,l,m,p,pp,rb,ru,v,y
TradingDay,,,,,,,,,,,,,,,,,,,,,
20180102,287494.0,390406.0,619616.0,278554.0,606340.0,476462.0,1039766.0,233118.0,602744.0,255972.0,...,185912.0,249926.0,576250.0,1964960.0,505862.0,548540.0,2413572.0,367234.0,326354.0,856442.0
20180103,295694.0,397376.0,615706.0,283108.0,608344.0,480132.0,1046946.0,233774.0,583810.0,269758.0,...,197770.0,287396.0,547130.0,2020708.0,476372.0,516452.0,2433636.0,374936.0,331772.0,865796.0
20180104,299046.0,389700.0,648542.0,279006.0,625648.0,492682.0,1246046.0,237044.0,595966.0,268078.0,...,193662.0,294114.0,539944.0,1995102.0,480464.0,516480.0,2581090.0,404120.0,344132.0,845190.0
20180105,304818.0,392936.0,619200.0,281020.0,614906.0,492040.0,1209508.0,235366.0,584178.0,271418.0,...,197094.0,266936.0,536522.0,2007274.0,472518.0,514038.0,2744436.0,405010.0,338704.0,831646.0
20180108,303366.0,469372.0,750748.0,281968.0,612182.0,490912.0,1359922.0,233268.0,595602.0,265332.0,...,222406.0,317868.0,527394.0,1965854.0,485334.0,520284.0,2717212.0,413072.0,341356.0,893520.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20191225,917972.0,339964.0,1570658.0,210082.0,514534.0,472068.0,1452640.0,155630.0,617078.0,218198.0,...,440222.0,166158.0,755098.0,3247966.0,1366834.0,687374.0,2762568.0,439418.0,443094.0,1227886.0
20191226,914188.0,339590.0,1441160.0,242088.0,551760.0,462454.0,1438828.0,154318.0,674884.0,219286.0,...,456524.0,184838.0,763304.0,3302810.0,1481316.0,650784.0,2821242.0,436794.0,449918.0,1264308.0
20191227,1081114.0,358280.0,1575878.0,240588.0,510512.0,521656.0,1515980.0,152552.0,684910.0,225266.0,...,480912.0,178598.0,761602.0,3304670.0,1610860.0,620994.0,2903738.0,438640.0,452762.0,1311608.0
